# Dechow's F-Score
## [Original Paper](https://poseidon01.ssrn.com/delivery.php?ID=556009073006105088000122086073105078024081022041024036031090115103120083014070085024099002021039108007045065097003086091003112104036001013036115108029089018008107000061062003088073093002097123117025122099081111118125007118076120069029016119120082027&EXT=pdf)


In [1]:
%pip install "calcbench-api-client[Pandas]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import calcbench as cb
import pandas as pd
import numpy as np

In [3]:
metrics = [
    "Assets",
    "Liabilities",
    "CurrentAssets",
    "Cash",
    "ShortTermInvestments",
    "CurrentLiabilities",
    "CurrentLongTermDebt",
    "LongTermDebt",
    "PreferredStockValue",
    "AccountsReceivable",
    "Inventory",
    "PPE",
    "NetIncome",
    "SharesIssued",
    "LoansReceivable",
    "LongTermInvestments",
    "Revenue",
]

In [4]:
tickers = cb.tickers(index="DJIA")

In [16]:
data = cb.standardized(
    company_identifiers=tickers, metrics=metrics, period_type="annual"
).fillna(0).unstack('metric')['value']

In [18]:
average_total_assets = (data["Assets"] + data.shift(1)["Assets"]) / 2

In [19]:
# Working Capital
WC = data["CurrentAssets"] - data["CurrentLiabilities"]

In [20]:
# Net Charge Offs
investments_and_advances = data[
    "LoansReceivable"
]  # This is probably not totally correct.  The paper says Investments and Advances is DATA 32 from CompuStat
NCO = (data["Assets"] - data["CurrentAssets"] - investments_and_advances) - (
    data["Liabilities"] - data["CurrentLiabilities"] - data["LongTermDebt"]
)

In [21]:
FIN = (data["ShortTermInvestments"] + data["LongTermInvestments"]) - (
    data["LongTermDebt"] + data["CurrentLongTermDebt"] + data["PreferredStockValue"]
)

In [22]:
RSST_Accruals = (WC.diff() + NCO.diff() + FIN.diff()) / average_total_assets

In [23]:
percent_soft_assets = (data["Assets"] - data["PPE"]) / data["Assets"]

In [24]:
issued_shares = data["SharesIssued"].diff() > 0

In [25]:
change_inventory = data["Inventory"].diff() / average_total_assets

In [26]:
change_receivables = data["AccountsReceivable"] / average_total_assets

In [27]:
cash_sales = data["Revenue"] - data["AccountsReceivable"].diff()

In [28]:
ROA = data["NetIncome"] / average_total_assets

In [29]:
intercept = -7.893

In [30]:
predicted_value = (
    intercept
    + (0.79 * RSST_Accruals)
    + (2.518 * change_receivables)
    + (1.191 * change_inventory)
    + (1.979 * percent_soft_assets)
    + (0.171 * cash_sales.pct_change())
    + (-0.932 * ROA.diff())
    + (1.029 * issued_shares)
)

C:\Users\andre\AppData\Local\Temp\ipykernel_22004\4123540390.py:7: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  + (0.171 * cash_sales.pct_change())


In [31]:
probability = (np.e ** predicted_value) / (1 + np.e ** predicted_value)